In [2]:
# Retrieve the ratemap data saved from previous code.
# Spatial information and
import os
import pickle
import numpy as np
import pandas as pd
import scipy.stats
from tqdm import tqdm
import pdb
import sklearn.preprocessing
import h5py

from maze_graph import *

# read in a key .xlsx file which contains information of all sessions and all mice.
file = pd.read_excel("D:\YSY\mice_maze_metadata_time_correction.xls", sheet_name = "training_recording_new")  

# set the total path for figure saving
totalpath = "D:\YSY"

# Check whether the file path is existed or not
def FindPathOfPRPC(Rec):
    totalpath = "D:\YSY"
    path = os.path.join(totalpath,str(Rec[0]),str(Rec[1]),"rate_map_pcrc.pkl")
    if os.path.exists(path):
        return path
    return "F"

# Check whether the file path is existed or not
def FindPathOfRawRate(Rec):
    totalpath = "D:\YSY"
    path = os.path.join(totalpath,str(Rec[0]),str(Rec[1]),"rate_map_all.pkl")
    if os.path.exists(path):
        return path
    return "F"

# Check whether the file path is existed or not
def FindPathOfTime(Rec):
    totalpath = "D:\YSY"
    path = os.path.join(totalpath,str(Rec[0]),str(Rec[1]),"SI_cal.pkl")
    if os.path.exists(path):
        return path
    return "F"

# Return the array of pkl files in certain days. "days" provide the certain day.
def GetFilePathCollection(maze_type = 1, training_days = 1, file = file):
    days = training_days
    # index of collections
    Col_ind = np.where((file['training_day'] == days) & (file['maze_type'] == maze_type))[0]
    if len(Col_ind) == 0:
        return [],[],[]
    
    # Files that store the behav/ms information of cells. [spikes_all, spike_nodes_all, t_total_all, t_nodes_frac_all]
    ms_file = []     
    # Files that store the place cells(pc)' data. [rate_map_pc, rate_map_pc_norm, rate_map_ordered, rate_map_rc, rate_map_rc_norm, is_runcell_isi, rate_map_rc_ordered]
    pc_file = []
    # Files that store the all cells(ac)' data. [is_placecell_isi, SI_all, rate_map_all]
    ac_file = []
    
    for i in range(len(Col_ind)):
        NumDate = [0,0]
        NumDate[0] = file['number'][Col_ind[i]]
        NumDate[1] = file['date'][Col_ind[i]]
        if FindPathOfPRPC(NumDate) == "F" or FindPathOfRawRate(NumDate) == "F" or FindPathOfTime(NumDate) == "F":
            print(NumDate[0],NumDate[1]," Meet a FILE LOSS ERROR!")
            if FindPathOfPRPC(NumDate) == "F":
                print(os.path.join(totalpath,str(NumDate[0]),str(NumDate[1]),"rate_map_pcrc.pkl   LOSS"))
            if FindPathOfRawRate(NumDate) == "F":
                print(os.path.join(totalpath,str(NumDate[0]),str(NumDate[1]),"rate_map_all.pkl    LOSS"))
            if FindPathOfTime(NumDate) == "F":
                print(os.path.join(totalpath,str(NumDate[0]),str(NumDate[1]),"SI_cal.pkl          LOSS"))
            continue
        pc_file.append(FindPathOfPRPC(NumDate))
        ac_file.append(FindPathOfRawRate(NumDate))
        ms_file.append(FindPathOfTime(NumDate))
    if len(pc_file) != len(ac_file) or len(pc_file) != len(ms_file) or len(ac_file) != len(ms_file):
        print("Warning! The dimension of Col_file and SI_file is not the same! That may throw errors in later analysis!")
    return pc_file, ac_file, ms_file

In [3]:
# data_type = 'pc'  -- place cells, 
#             'ac'  -- all cells,
#             'npc' -- nonplace cells,
#             'pcn' -- place cell (normed),
#             'aSI'  -- Spatial information for all cells
#             'pSI'  -- Spatial information for place cells
#             'npSI' -- Spatial information for nonplace cells
#             'ispc_isi' -- is_placecll_isi
#             'spikes' -- Spatial information for nonplace cells
#             'spike_nodes' -- is_placecll_isi
#             't_total' -- Spatial information for nonplace cells
#             't_nodes_frac' -- is_placecll_isi
#             'aWPF'  -- WPFmap_allmice

def Combination(data_type = 'pc',training_days = 1, maze_type = 1):
    pc_file, ac_file, ms_file = GetFilePathCollection(file = file, training_days = training_days, maze_type = maze_type)
    
    # 'pc'  -- place cells firing rate, 'pcn' -- place cell firing rate (normed)。
    if data_type == 'pc' or data_type == 'pcn':
        data = []
        for i in range(len(pc_file)):
            if os.path.getsize(pc_file[i]) > 0:
                with open(pc_file[i],'rb') as handle:
                    pc_data = pickle.load(handle)
            else:
                print(pc_file[i],"is not existed!")
            if data_type == 'pc':
                for j in range(len(pc_data[0])):
                    data.append(pc_data[0][j])
            else:
                for j in range(len(pc_data[1])):
                    data.append(pc_data[1][j])
        return data
    
    # 'ac'  -- all cells,
    elif data_type == 'ac':
        data = []
        for i in range(len(ac_file)):
            if os.path.getsize(ac_file[i]) > 0:
                with open(ac_file[i],'rb') as handle:
                    ac_data = pickle.load(handle)
            else:
                print(ac_file[i],"is not existed!")
            for j in range(len(ac_data[2])):
                data.append(ac_data[2][j])
        return data
    
    # 'npc' -- nonplace cells,
    elif data_type == 'npc':
        data = []
        for i in range(len(ac_file)):
            if os.path.getsize(ac_file[i]) > 0:
                with open(ac_file[i],'rb') as handle:
                    ac_data = pickle.load(handle)
            else:
                print(ac_file[i],"is not existed!")
            for j in range(len(ac_data[2])):
                if ac_data[0][j] == 0: # is_placecell_isi[j] == 0 -> non place cell
                    data.append(ac_data[2][j])
        return data

    # 'aSI'  -- SI of all cells,
    elif data_type == 'aSI':
        data = []
        for i in range(len(ac_file)):
            if os.path.getsize(ac_file[i]) > 0:
                with open(ac_file[i],'rb') as handle:
                    ac_data = pickle.load(handle)
            else:
                print(ac_file[i],"is not existed!")
            for j in range(len(ac_data[1])):
                data.append(ac_data[1][j])
        return data
    
    # 'pSI'  --SI of place cells
    elif data_type == 'pSI':
        data = []
        for i in range(len(ac_file)):
            if os.path.getsize(ac_file[i]) > 0:
                with open(ac_file[i],'rb') as handle:
                    ac_data = pickle.load(handle)
            else:
                print(ac_file[i],"is not existed!")
            for j in range(len(ac_data[1])):
                if ac_data[0][j] == 1: # is_placecell_isi[j] == 0 -> non place cell
                    data.append(ac_data[1][j])
        return data
    
    # 'npSI'  --SI of nonplace cells
    elif data_type == 'npSI':
        data = []
        for i in range(len(ac_file)):
            if os.path.getsize(ac_file[i]) > 0:
                with open(ac_file[i],'rb') as handle:
                    ac_data = pickle.load(handle)
            else:
                print(ac_file[i],"is not existed!")
            for j in range(len(ac_data[1])):
                if ac_data[0][j] == 0: # is_placecell_isi[j] == 0 -> non place cell
                    data.append(ac_data[1][j])
        return data    
    
    # 'ispc_isi'  -- is_placecell_isi
    elif data_type == 'ispc_isi':
        data = []
        for i in range(len(ac_file)):
            if os.path.getsize(ac_file[i]) > 0:
                with open(ac_file[i],'rb') as handle:
                    ac_data = pickle.load(handle)
            else:
                print(ac_file[i],"is not existed!")
            for j in range(len(ac_data[0])):
                data.append(ac_data[0][j])
        return data
    
    # 'spikes'  -- spikes of all neurons, combined all neurons (in different mice)
    elif data_type == 'spikes':
        data = []
        for i in range(len(ms_file)):
            if os.path.getsize(ms_file[i]) > 0:
                with open(ms_file[i],'rb') as handle:
                    ms_data = pickle.load(handle)
            else:
                print(ms_file[i],"is not existed!")
            for j in range(len(ms_data[0])):
                data.append(ms_data[0][j])
        return data

    # 'spike_nodes'  -- spike_nodes of each spike, combined all neurons (in different mice)
    elif data_type == 'spike_nodes':
        data = []
        for i in range(len(ms_file)):
            if os.path.getsize(ms_file[i]) > 0:
                with open(ms_file[i],'rb') as handle:
                    ms_data = pickle.load(handle)
            else:
                print(ms_file[i],"is not existed!")
            for j in range(len(ms_data[1])):
                data.append(ms_data[1][j])
        return data
    
    # 't_total'  -- t_total of each session, combined all sessions
    elif data_type == 't_total':
        data = []
        for i in range(len(ms_file)):
            if os.path.getsize(ms_file[i]) > 0:
                with open(ms_file[i],'rb') as handle:
                    ms_data = pickle.load(handle)
            else:
                print(ms_file[i],"is not existed!")
            for j in range(len(ms_data[2])):
                data.append(ms_data[2][j])
        return data
    
    # 't_nodes_frac'  -- t_nodes_frac of each session, combined all sessions
    elif data_type == 't_nodes_frac':
        data = []
        for i in range(len(ms_file)):
            if os.path.getsize(ms_file[i]) > 0:
                with open(ms_file[i],'rb') as handle:
                    ms_data = pickle.load(handle)
            else:
                print(ms_file[i],"is not existed!")
            for j in range(len(ms_data[3])):
                data.append(ms_data[3][j])
        return data
    
    # WPFmap_allmice for place cells
    elif data_type[0:7] == 'aWPF_pc':
        rate_map_all = Get_split_FRdata(training_days = training_days,maze_type = maze_type, data_type = 'pc',path_type = 'ap')
        WPFmap_all = []
        for i in range(len(rate_map_all)):
            WPFmap = Generate_WholePlaceFieldMap(rate_map_all[i],maze_type)
            if data_type[8::] in ['','c','C','co','Co','cor','Cor','Core','core']:
                WPFmap = BubbleSortWithcore(WPFmap, rate_map_all[i])
            elif data_type[8::] in ['l','L','len','Len','le','Le','Length','length','lent','Lent','lenth','Lenth']:
                WPFmap = BubbleSortWithLength(WPFmap)
            elif data_type[8::] in ['d','D','Dist','dist','distance','Distance','di','Di','Dis','dis']:
                WPFmap = BubbleSortWithDistance(WPFmap, maze_type = maze_type)
            else:
                print("only 'c','C','co','Co','cor','Cor','Core','core','l','L','len','Len','le','Le','Length','length','lent','Lent','lenth','Lenth','d','D','Dist','dist','distance','Distance','di','Di','Dis','dis' are all legal.")
                print("Do you mean not sort the WPFmap?")
            WPFmap_all.append(WPFmap)
        return WPFmap_all
    
    # WPFmap_allmice for nonplace cells
    elif data_type[0:8] == 'aWPF_npc':
        rate_map_all = Get_split_FRdata(training_days = training_days,maze_type = maze_type, data_type = 'npc',path_type = 'ap')
        WPFmap_all = []
        for i in range(len(rate_map_all)):
            WPFmap = Generate_WholePlaceFieldMap(rate_map_all[i],maze_type)
            if data_type[9::] in ['','c','C','co','Co','cor','Cor','Core','core']:
                WPFmap = BubbleSortWithcore(WPFmap, rate_map_all[i])
            elif data_type[9::] in ['l','L','len','Len','le','Le','Length','length','lent','Lent','lenth','Lenth']:
                WPFmap = BubbleSortWithLength(WPFmap)
            elif data_type[9::] in ['d','D','Dist','dist','distance','Distance','di','Di','Dis','dis']:
                WPFmap = BubbleSortWithDistance(WPFmap, maze_type = maze_type)
            else:
                print("only 'c','C','co','Co','cor','Cor','Core','core','l','L','len','Len','le','Le','Length','length','lent','Lent','lenth','Lenth','d','D','Dist','dist','distance','Distance','di','Di','Dis','dis' are all legal.")
                print("Do you mean not sort the WPFmap?")
            WPFmap_all.append(WPFmap)
        return WPFmap_all
    
    # WPFmap_allmice for all cells
    elif data_type[0:7] == 'aWPF_ac':
        rate_map_all = Get_split_FRdata(training_days = training_days,maze_type = maze_type, data_type = 'adc',path_type = 'ap')
        WPFmap_all = []
        for i in range(len(rate_map_all)):
            WPFmap = Generate_WholePlaceFieldMap(rate_map_all[i],maze_type)
            if data_type[8::] in ['','c','C','co','Co','cor','Cor','Core','core']:
                WPFmap = BubbleSortWithcore(WPFmap, rate_map_all[i])
            elif data_type[8::] in ['l','L','len','Len','le','Le','Length','length','lent','Lent','lenth','Lenth']:
                WPFmap = BubbleSortWithLength(WPFmap)
            elif data_type[8::] in ['d','D','Dist','dist','distance','Distance','di','Di','Dis','dis']:
                WPFmap = BubbleSortWithDistance(WPFmap, maze_type = maze_type)
            else:
                print("only 'c','C','co','Co','cor','Cor','Core','core','l','L','len','Len','le','Le','Length','length','lent','Lent','lenth','Lenth','d','D','Dist','dist','distance','Distance','di','Di','Dis','dis' are all legal.")
                print("Do you mean not sort the WPFmap?")
            WPFmap_all.append(WPFmap)
        return WPFmap_all

    else:
        print("Invalid data_type! Please check your spelling! only value 'pc','ac','npc','pcn','si_all','spikes','spike_nodes','t_total','t_nodes_frac' are available!")
        return []
    
# Clean the NaN value
def Clear_Ratemap_NaN(rate_map_all):
    for j in range(len(rate_map_all)):
        for k in range(np.shape(rate_map_all)[1]):
            if mt.isnan(rate_map_all[j][k]):
                rate_map_all[j][k] = 0.0
    return rate_map_all

def calc_SI_path(training_days = 1, maze_type = 1, path_type = 'cp'):
    # SI in total path is no need to calculate but could be called in files directly
    if path_type == 'ap' or path_type == 'no':
        print("  Also you can get the answer, however we strongly recommend you to use function Combination() directly!")
        return Combination(training_days=training_days, maze_type=maze_type,data_type='aSI')
    
    spikes, spike_nodes, t_total, t_nodes_frac = Get_split_msdata(training_days = training_days, maze_type = maze_type, data_type = 'spikes',path_type=path_type)
    SI = np.zeros(len(spikes))
    for i in range(len(spikes)):
        SI[i] = calc_SI(spikes[i], spike_nodes[i], t_total[i], t_nodes_frac[i])
    return SI

# spatial information (for one neuron) 
# see DOI: 10.1126/science.aav9199
# mean_rate = sum(spikes)/t_total # mean firing rate
def calc_SI(spikes, rate_map, t_total, t_nodes_frac):
    mean_rate = sum(spikes)/t_total # mean firing rate
    logArg = rate_map / mean_rate;
    logArg[logArg == 0] = 1; # keep argument in log non-zero

    IC = np.nansum(t_nodes_frac * rate_map * np.log2(logArg)) # information content
    SI = IC / mean_rate; # spatial information (bits/spike)
    return(SI)

In [4]:
# split data in correct path, incorrect path or reorder them in a ranked order.
# path_type : 'ap' (all path in certain order), 'cp' (correct path), 'icp'(incorrect path), 'no' (not orderred all path)

# data_type = 'pc'  -- place cells, 
#             'ac'  -- all cells,
#             'npc' -- nonplace cells,
#             'pcn' -- place cell (normed),
#             'aSI'  -- Spatial information for all cells
#             'pSI'  -- Spatial information for place cells
#             'npSI' -- Spatial information for nonplace cells
#             'ispc_isi' -- is_placecll_isi
#             'spikes' -- Spatial information for nonplace cells
#             'spike_nodes' -- is_placecll_isi
#             't_total' -- Spatial information for nonplace cells
#             't_nodes_frac' -- is_placecll_isi

# split ms/behav data with different path
def Get_split_msdata(data_type = 'spikes',training_days = 1, maze_type = 1, path_type = 'ap', GetFourResults = 1):
    # Get four necessary variable matrices.
    bef_spikes = Combination(data_type = 'spikes', training_days = training_days, maze_type = maze_type)
    bef_spike_nodes = Combination(data_type = 'spike_nodes', training_days = training_days, maze_type = maze_type)
    bef_t_total = Combination(data_type = 't_total', training_days = training_days, maze_type = maze_type)
    bef_t_nodes_frac = Combination(data_type = 't_nodes_frac', training_days = training_days, maze_type = maze_type)
    
    correct_path = CorrectPath_maze_1 if maze_type == 1 else CorrectPath_maze_2
    incorrect_path = IncorrectPath_maze_1 if maze_type == 1 else IncorrectPath_maze_2
    reorderred_path = correct_path + incorrect_path
    
    if path_type in ['no','ap']:
        if GetFourResults == 1:
            if data_type in ['spikes','spike_nodes','t_total','t_nodes_frac']:
                return [bef_spikes,bef_spike_nodes,bef_t_total,bef_t_nodes_frac]
            elif data_type in ['pc','ac','npc']:
                print("WARNING!!! You should use another funciton Get_split_FRdata() to deel with data_type '"+str(data_type)+"'")
                print("Here we help you to call this funciton. However we strongly recommend you to correct it.")
                return Get_split_FRdata(data_type = data_type,training_days = training_days, maze_type = maze_type, path_type = path_type)
            else:
                print("WARNING!!! Type Error !!! data_type '",data_type,"' cannot be split into correct path!")
                return []
            
        else:
            if data_type == 'spikes':
                return bef_spikes
            elif data_type == 'spike_nodes':
                return bef_spike_nodes
            elif data_type == 't_total':
                return bef_t_total
            elif data_type == 't_nodes_frac':
                return bef_t_nodes_frac
            elif data_type in ['pc','ac','npc']:
                print("WARNING!!! You should use another funciton Get_split_FRdata() to deel with data_type '"+str(data_type)+"'")
                print("Here we help you to call this funciton. However we strongly recommend you to correct it.")
                return Get_split_FRdata(data_type = data_type,training_days = training_days, maze_type = maze_type, path_type = path_type)
            else:
                print("WARNING!!! Type Error !!! data_type '",data_type,"' cannot be split into correct path!")
                return []    
    
    elif path_type == 'cp':
        print('path_type = '+path_type)
        if data_type in ['pc','ac','npc']:
            print("WARNING!!! You should use another funciton Get_split_FRdata() to deel with data_type '"+str(data_type)+"'")
            print("Here we help you to call this funciton. However we strongly recommend you to correct it.")
            return Get_split_FRdata(data_type = data_type,training_days = training_days, maze_type = maze_type, path_type = path_type)
        
        elif data_type in ['spikes','spike_nodes','t_total','t_nodes_frac']:
            print("data_type: "+data_type)
            aft_spikes = []
            aft_spike_nodes = []
            aft_t_total = np.zeros_like(bef_t_total)    # n_neuron_all * 1
            aft_t_nodes_frac = np.zeros((len(bef_t_total),len(correct_path)))  # n_neuron_all * 144(111/xxx)
            if len(bef_t_total) != len(bef_t_nodes_frac):
                print("WARNING!!! the dimension of t_total is not equal to the dimension of t_nodes_frac_all, that may cause ERROR in later analysis!")
            
            print("began to calculate time")
            for i in range(len(bef_t_total)):
                for k in range(len(correct_path)):
                    aft_t_nodes_frac[i][k] = bef_t_nodes_frac[i][correct_path[k]-1]
                    if np.isnan(bef_t_nodes_frac[i][correct_path[k]-1] * bef_t_total[i]):
                        continue
                    aft_t_total[i] += bef_t_nodes_frac[i][correct_path[k]-1] * bef_t_total[i]   
            aft_t_nodes_frac = Norm_t_nodes_frac_part(aft_t_nodes_frac)
            
            print("began to calculate spikes")
            for i in range(len(bef_spikes)):
                ind = [list(np.where(np.array(bef_spike_nodes[i]) == correct_path[j])[0]) for j in range(len(correct_path))]
                merge_ind = sum(ind,[])
                co_spikes = [bef_spikes[i][k] for k in merge_ind]
                co_spike_nodes = [bef_spike_nodes[i][k] for k in merge_ind]

                aft_spike_nodes.append(co_spike_nodes)
                aft_spikes.append(co_spikes)
        
            print("Finish calculating!")
            
            if GetFourResults == 1:
                return [aft_spikes,aft_spike_nodes,aft_t_total,aft_t_nodes_frac]
            else:
                if data_type == 'spikes':
                    return aft_spikes
                elif data_type == 'spike_nodes':
                    return aft_spike_nodes
                elif data_type == 't_total':
                    return aft_t_total
                elif data_type == 't_nodes_frac':
                    return aft_t_nodes_frac
                else:
                    print("ERROR!")
                    return []
        
        else:
            print("WARNING!!! Type Error !!! data_type '",data_type,"' cannot be split into correct path!")
            return []
        
    elif path_type == 'icp':
        if data_type in ['pc','ac','npc']:
            print("WARNING!!! You should use another funciton Get_split_FRdata() to deel with data_type '"+str(data_type)+"'")
            print("Here we help you to call this funciton. However we strongly recommend you to correct it.")
            return Get_split_FRdata(data_type = data_type,training_days = training_days, maze_type = maze_type, path_type = path_type)
        
        elif data_type in ['spikes','spike_nodes','t_total','t_nodes_frac']:
            aft_spikes = []
            aft_spike_nodes = []
            aft_t_total = np.zeros_like(bef_t_total)    # n_neuron_all * 1
            aft_t_nodes_frac = np.zeros((len(bef_t_total),len(incorrect_path)))  # n_neuron_all * 144(111/xxx)
            if len(bef_t_total) != len(bef_t_nodes_frac):
                print("WARNING!!! the dimension of t_total is not equal to the dimension of t_nodes_frac_all, that may cause ERROR in later analysis!")
            
            for i in range(len(bef_t_total)):
                for k in range(len(incorrect_path)):
                    aft_t_nodes_frac[i][k] = bef_t_nodes_frac[i][incorrect_path[k]-1]
                    if np.isnan(bef_t_nodes_frac[i][incorrect_path[k]-1] * bef_t_total[i]):
                        continue
                    aft_t_total[i] += bef_t_nodes_frac[i][incorrect_path[k]-1] * bef_t_total[i]   
            aft_t_nodes_frac = Norm_t_nodes_frac_part(aft_t_nodes_frac)
            
            for i in range(len(bef_spikes)):
                ind = [list(np.where(np.array(bef_spike_nodes[i]) == incorrect_path[j])[0]) for j in range(len(incorrect_path))]
                merge_ind = sum(ind,[])
                inco_spikes = [bef_spikes[i][k] for k in merge_ind]
                inco_spike_nodes = [bef_spike_nodes[i][k] for k in merge_ind]

                aft_spike_nodes.append(inco_spike_nodes)
                aft_spikes.append(inco_spikes)
            
            if GetFourResults == 1:
                return [aft_spikes,aft_spike_nodes,aft_t_total,aft_t_nodes_frac]
            else:
                if data_type == 'spikes':
                    return aft_spikes
                elif data_type == 'spike_nodes':
                    return aft_spike_nodes
                elif data_type == 't_total':
                    return aft_t_total
                elif data_type == 't_nodes_frac':
                    return aft_t_nodes_frac
                else:
                    print("ERROR!")
                    return []
        
        else:
            print("WARNING!!! Type Error !!! data_type '",data_type,"' cannot be split into correct path!")
            return []
    
    else:
        print("Invalid path_type! Please check your spelling! only value 'spikes','in','ap','no' are acceptable!")
        return []        
              
              
#---------------------------------------------------------------------------------------------------------------------------------------
# split firing rate data with different path
def Get_split_FRdata(data_type = 'pc',training_days = 1, maze_type = 1, path_type = 'ap'):
    bef_data = Combination(data_type = data_type, training_days = training_days, maze_type = maze_type)
    
    correct_path = CorrectPath_maze_1 if maze_type == 1 else CorrectPath_maze_2
    incorrect_path = IncorrectPath_maze_1 if maze_type == 1 else IncorrectPath_maze_2
    reorderred_path = correct_path + incorrect_path
    
    if path_type == 'no':
        bef_data = Clear_Ratemap_NaN(bef_data)
        return bef_data
    
    elif path_type == 'cp':
        if data_type in ['spikes','spike_nodes','t_total','t_nodes_frac']:
            print("WARNING!!! You should use another funciton Get_split_msdata() to deel with data_type '"+str(data_type)+"'")
            print("Here we help you to call this funciton. However we strongly recommend you to correct it.")
            return Get_split_msdata(data_type = data_type,training_days = training_days, maze_type = maze_type, path_type = path_type)
        
        elif data_type in ['pc','ac','npc']:
            bef_data = Clear_Ratemap_NaN(bef_data)
            aft_data = np.zeros((len(bef_data),len(correct_path)))
            for i in range(len(bef_data)):
                for j in range(len(aft_data[0])):
                    aft_data[i][j] = bef_data[i][correct_path[j]-1]
            print("Get_split_FRdata: ",np.shape(aft_data))
            return aft_data
        
        else:
            print("WARNING!!! Type Error !!! data_type '",data_type,"' cannot be split into correct path!")
            return []
        
    elif path_type == 'icp':
        if data_type in ['spikes','spike_nodes','t_total','t_nodes_frac']:
            print("WARNING!!! You should use another funciton Get_split_msdata() to deel with data_type '"+str(data_type)+"'")
            print("Here we help you to call this funciton. However we strongly recommend you to correct it.")
            return Get_split_msdata(data_type = data_type,training_days = training_days, maze_type = maze_type, path_type = path_type)
        
        elif data_type in ['pc','ac','npc']:
            bef_data = Clear_Ratemap_NaN(bef_data)
            aft_data = np.zeros((len(bef_data),len(incorrect_path)))
            for i in range(len(bef_data)):
                for j in range(len(aft_data[0])):
                    aft_data[i][j] = bef_data[i][incorrect_path[j]-1]
            print("Get_split_FRdata: ",np.shape(aft_data))
            return aft_data
        
        else:
            print("WARNING!!! Type Error !!! data_type '",data_type,"' cannot be split into correct path!")
            return []
        
    elif path_type == 'ap':
        if data_type in ['spikes','spike_nodes','t_total','t_nodes_frac']:
            print("WARNING!!! You should use another funciton Get_split_msdata() to deel with data_type '"+str(data_type)+"'")
            print("Here we help you to call this funciton. However we strongly recommend you to correct it.")
            return Get_split_msdata(data_type = data_type,training_days = training_days, maze_type = maze_type, path_type = path_type)
        
        elif data_type in ['pc','ac','npc']:
            bef_data = Clear_Ratemap_NaN(bef_data)
            aft_data = np.zeros((len(bef_data),len(reorderred_path)))
            for i in range(len(bef_data)):
                for j in range(len(aft_data[0])):
                    aft_data[i][j] = bef_data[i][reorderred_path[j]-1]
            print("Get_split_FRdata: ",np.shape(aft_data))
            return aft_data
        
        else:
            print("WARNING!!! Type Error !!! data_type '",data_type,"' cannot be split into correct path!")
            return []
    
    else:
        print("Invalid path_type! Please check your spelling! only value 'cp','icp','ap','no' are acceptable!")
        return []        

#---------------------------------------------------------------------------------------------------------------------------------------
# split spatial information with different path   
def Get_Split_SIdata(training_days = 1, maze_type = 1, data_type = 'pc'):
    # Devide the data into two groups: the correct path group and the incorrect path group
    msdata_co = Get_split_msdata(training_days = training_days, maze_type = maze_type, data_type = 'spikes', path_type = 'cp')
    ratemap_co = Get_split_FRdata(training_days = training_days, maze_type = maze_type, data_type = 'ac', path_type = 'cp')
    msdata_inc = Get_split_msdata(training_days = training_days, maze_type = maze_type, data_type = 'spikes', path_type = 'icp')
    ratemap_inc = Get_split_FRdata(training_days = training_days, maze_type = maze_type, data_type = 'ac', path_type = 'icp')
    
    aSI = Combination(training_days = training_days, maze_type = maze_type, data_type = 'aSI')
    pSI = Combination(training_days = training_days, maze_type = maze_type, data_type = 'pSI')
    npSI = Combination(training_days = training_days, maze_type = maze_type, data_type = 'npSI')
    is_placecell_isi_all = Combination(training_days = training_days, maze_type = maze_type, data_type = 'ispc_isi')
    
    SI_co_all = np.zeros(len(is_placecell_isi_all))
    SI_inc_all = np.zeros(len(is_placecell_isi_all))
    for i in range(len(is_placecell_isi_all)):
        SI_co_all[i] = calc_SI(msdata_co[0][i],ratemap_co[i],msdata_co[2][i],msdata_co[3][i])
        SI_inc_all[i] = calc_SI(msdata_inc[0][i],ratemap_inc[i],msdata_inc[2][i],msdata_inc[3][i])
    
    if data_type == 'pc':
        pc_SI_co = []
        pc_SI_inc = []
        for i in range(len(is_placecell_isi_all)):
            if is_placecell_isi_all[i] == 1:
                pc_SI_co.append(SI_co_all[i])
                pc_SI_inc.append(SI_inc_all[i])
        return pSI, pc_SI_co, pc_SI_inc
    
    elif data_type == 'npc':
        npc_SI_co = []
        npc_SI_inc = []
        for i in range(len(is_placecell_isi_all)):
            if is_placecell_isi_all[i] == 0:
                npc_SI_co.append(SI_co_all[i])
                npc_SI_inc.append(SI_inc_all[i])
        return npSI, npc_SI_co, npc_SI_inc        
    
    elif data_type == 'ac':
        return aSI, SI_co_all, SI_inc_all
    
    else:
        print("Invalid path_type! Please check your spelling! only value 'pc','npc','ac' are acceptable!")
        return aSI, SI_co_all, SI_inc_all

    
def Norm_t_nodes_frac_part(t_nodes_frac):
    t_nodes_frac_norm = np.zeros_like(t_nodes_frac)
    for i in range(len(t_nodes_frac_norm)):
        sum_all = np.nansum(t_nodes_frac[i])
        for j in range(len(t_nodes_frac_norm[i])):
            if np.isnan(t_nodes_frac[i][j]):
                t_nodes_frac_norm[i][j] = np.nan
            else:
                t_nodes_frac_norm[i][j] = t_nodes_frac[i][j]/sum_all
    return t_nodes_frac_norm

def Norm_ratemap(rate_map_all):
    rate_map_all_norm = sklearn.preprocessing.minmax_scale(rate_map_all, feature_range=(0, 1), axis=1, copy=True)
    return rate_map_all_norm

In [ ]:
# correct path selective index (CPSI): (FR_correct - FR_incorrect)/(FR_correct + FR_incorrect)
# spatial information (for one neuron) 

# Calculate CPSI
def calc_CPSI(spikes, spike_nodes, t_total, t_nodes_frac, maze_type = 1):
    co_path = CorrectPath_maze_1 if maze_type == 1 else CorrectPath_maze_2
    inc_path = IncorrectPath_maze_1 if maze_type else IncorrectPath_maze_2
    
    # time used for correct path and incorrect path, either.
    t_total_co = t_total * np.nansum([t_nodes_frac[i-1] for i in co_path])
    t_total_inc = t_total * np.nansum([t_nodes_frac[i-1] for i in inc_path])
    
    # check if t_total_co + t_total_inc = t_total
    if t_total_co + t_total_inc - t_total > 0.0001 or t_total_co + t_total_inc - t_total < -0.0001:
        print("WARNING!!! t_total_co, t_total_inc calculate error!")
        print("///",t_total_co,"+",t_total_inc,'=?',t_total)
    
    # spikes number of each bin in correct path and incorrect path, either.
    co_count = [np.nansum(np.where((spike_nodes == co_path[i])&(spikes == 1))[0]) for i in range(len(co_path))]
    correct_spikes_number = np.nansum(co_count) # total spikes number in correct path
    inc_count = [np.nansum(np.where((spike_nodes == inc_path[i])&(spikes == 1))[0]) for i in range(len(inc_path))]
    incorrect_spikes_number = np.nansum(inc_count) # total spikes number in incorrect path
    
    
    # mean firing rate.
    meanFR_co = correct_spikes_number / t_total_co
    meanFR_inc = incorrect_spikes_number / t_total_inc
    #print("co_spikes:",correct_spikes_number)
    #print("in_spikes:",incorrect_spikes_number)
    #print("meanFR_co:",meanFR_co)
    #print("meanFR_in:",meanFR_inc)
    
    CPSI = (meanFR_co - meanFR_inc)/(meanFR_co + meanFR_inc)
    #print("CPSI:     ",CPSI)
    return CPSI

# 1000 times isi shuffle
def isCorrectPathEncodingCell(spikes, spike_nodes, t_total, t_nodes_frac,maze_type = 1):
    CPSI = calc_CPSI(spikes=spikes, spike_nodes=spike_nodes, t_total=t_total, t_nodes_frac=t_nodes_frac, maze_type = maze_type)
    
    _nbins = 144
    _coords_range = [0, _nbins +0.0001 ]
    # shuffle information for 1000 times
    shuffle_n = 1000
    CPSI_rand = np.zeros(shuffle_n)
    for i in range(shuffle_n):
        shuffle_tmp = np.random.randint(len(spikes))
        spikes_rand = np.roll(spikes, shuffle_tmp)
        spike_freq_rand, _, _ = scipy.stats.binned_statistic(
            spike_nodes,
            spikes_rand,
            bins=_nbins,
            statistic="sum",
            range=_coords_range)
        CPSI_rand[i] = calc_CPSI(spikes = spikes_rand, spike_nodes = spike_nodes, t_total = t_total, t_nodes_frac=t_nodes_frac, maze_type = maze_type)
    CPSI_thres = np.percentile(CPSI_rand, 95)
    if CPSI < CPSI_thres:
        print("  This cell is rejected by 95% threshold, i.e. does not encode correct path." )
        print("CPSI:     ",CPSI)
        return 0, CPSI
    else:
        print("  Ye! This cell encodes correct path!")
        print("CPSI:     ",CPSI)
        return 1, CPSI

# Generate all neuron's (combine all mice data together) selective information and save into 2 lists: 
# the 'is_CPEC_isi' which consists of values 0,1, only. 0 represents this neuron is not a CPEC, while 1 represents this neuron is CPEC
# the 'CPSI_all' which saves the absolute value of CPSI calculated by function calc_CPSI()
def Get_CPEC_1day(training_days = 1, maze_type = 1):
    
    # Get_split_msdata(training_days, maze_type, data_type,path_type) is used to generate a list of spikes, spike_nodes, t_total
    # t_nodes_frac of all neurons that have been detected in 1 certain day(training_days).
    # This function has been tested for many times and no bug is found, so you can use it directly.
    spikes, spike_nodes, t_total, t_nodes_frac = Get_split_msdata(training_days = training_days, maze_type = maze_type, data_type = 'spikes',path_type = 'ap')
    
    
    is_CPEC_isi = np.zeros(len(spikes))
    CPSI_all = np.zeros(len(spikes))
    
    # In place cells identificaiton there're some cases (for example, mice 10019, date 4_20, neuron 23) that found some 
    # 'silent neurons' which has indistinct firing activity and as a result, the size of spike_ind is 0 and could not be able 
    # to used for further shuffling. 
    # So, whenever the size of spike_ind is equal to zero, stop shuffling and return 0, because the silent neuron 
    # is actually neither a place cell nor a correct path/incorrect path encoding cell.
    SilentCell = []
    isSilent = np.zeros(len(spikes))
    for i in range(len(spikes)):
        isSilent[i] = isSilentNeuron(spikes[i][:])
        if isSilent[i] == 0:
            SilentCell.append(i)
            
    for i in tqdm(range(len(spikes))):
        # So whenever meet a silent cell, skip it and remain its is_CPEC_isi value and CPSI_all value as 0.
        if i in SilentCell:
            continue
        print(str(i+1)+"/"+str(len(spikes))+" ",end='')
        is_CPEC_isi[i], CPSI_all[i] = isCorrectPathEncodingCell(spikes[i], spike_nodes[i], t_total[i], t_nodes_frac[i],maze_type = maze_type)
    print(is_CPEC_isi, CPSI_all)
    
    return is_CPEC_isi, CPSI_all

# Judge whether a neuron is 'silent' from its spike sequence.
def isSilentNeuron(spikes):
    spike_ind = np.where(spikes==1)[0] # spike index
    if np.shape(spike_ind)[0] == 0:
        return 0
    else:
        return 1

# Generate CPEC information of all training days(9 days for maze 1 and 8 days for maze 2) and save them as a pickle file
# because the CPEC information generation costs too much times and it seems really necessary to avoiding running generation codes
# again.
def Get_CPEC_Allday():
    is_CPEC_isi_all_maze1 = []
    is_CPEC_isi_all_maze2 = []
    CPSI_all_maze1 = []
    CPSI_all_maze2 = []
    for i in range(0,9):
        is_CPEC_isi, CPSI_all = Get_CPEC_1day(training_days = i+1, maze_type = 1)
        is_CPEC_isi_all_maze1.append(is_CPEC_isi)
        CPSI_all_maze1.append(CPSI_all)

    for i in range(0,8):
        is_CPEC_isi, CPSI_all = Get_CPEC_1day(training_days = i+1, maze_type = 2)
        is_CPEC_isi_all_maze2.append(is_CPEC_isi)
        CPSI_all_maze2.append(CPSI_all)        
        
    path = os.path.join(totalpath,"all_mice_data_maze\CPEC.pkl")
    with open(path, 'wb') as f:
        pickle.dump([is_CPEC_isi_all_maze1,CPSI_all_maze1,is_CPEC_isi_all_maze2,CPSI_all_maze2], f)
    
Get_CPEC_Allday()

10027 20200417  Meet a FILE LOSS ERROR!
D:\YSY\10027\20200417\rate_map_pcrc.pkl   LOSS
D:\YSY\10027\20200417\rate_map_all.pkl    LOSS
D:\YSY\10027\20200417\SI_cal.pkl          LOSS
10027 20200417  Meet a FILE LOSS ERROR!
D:\YSY\10027\20200417\rate_map_pcrc.pkl   LOSS
D:\YSY\10027\20200417\rate_map_all.pkl    LOSS
D:\YSY\10027\20200417\SI_cal.pkl          LOSS
10027 20200417  Meet a FILE LOSS ERROR!
D:\YSY\10027\20200417\rate_map_pcrc.pkl   LOSS
D:\YSY\10027\20200417\rate_map_all.pkl    LOSS
D:\YSY\10027\20200417\SI_cal.pkl          LOSS
10027 20200417  Meet a FILE LOSS ERROR!
D:\YSY\10027\20200417\rate_map_pcrc.pkl   LOSS
D:\YSY\10027\20200417\rate_map_all.pkl    LOSS
D:\YSY\10027\20200417\SI_cal.pkl          LOSS


  0%|                                                                                          | 0/665 [00:00<?, ?it/s]

1/665 

  0%|                                                                                | 1/665 [00:06<1:14:03,  6.69s/it]

  This cell is rejected by 95% threshold, i.e. does not encode correct path.
CPSI:      -0.07586808395185211
2/665 

...

  This cell is rejected by 95% threshold, i.e. does not encode correct path.
CPSI:      0.07875854717650757
95/665 

 14%|███████████▎                                                                   | 95/665 [13:00<2:35:43, 16.39s/it]

  This cell is rejected by 95% threshold, i.e. does not encode correct path.
CPSI:      0.20809883794427417
96/665 

 14%|███████████▍                                                                   | 96/665 [13:17<2:35:46, 16.43s/it]

  Ye! This cell encodes correct path!
CPSI:      0.4906101529541525
97/665 